In [ ]:
import numpy as np
import scipy.signal as ss
import matplotlib.pyplot as plt
from scipy import interpolate
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from obspy.signal.filter import bandpass

plt.rcParams["font.family"] = "Times"
plt.rcParams.update({'font.size': 50})
plt.rcParams['xtick.major.pad']='12'
plt.rcParams['ytick.major.pad']='12'

In [ ]:
# Medium setup ======================================

# Radius of receiver sphere [m].
R=0.1
# Radius of source sphere [m].
R_src=0.15
# Reference velocity for synthetics [m/s].
c_syn=1500.0
# Velocity for observations [m/s].
c_obs=1550.0


# Random sources. ===================================

# Number of sources.
nsrc=10000
# Number of source realisations.
N=200
# Random seed for reproducibility.
np.random.seed(0)


# Time series and filtering. ========================

# Length of one time series [s].
T=200.0*R/c_syn
# Number of samples in the time series.
n=int(300*T*c_syn/R)
# Frequency band [Hz].
freqmin=75.0e3
freqmax=400.0e3

print('transit time: %g ms' % (2000.0*R/c_syn))
print('total length of time series: %g s' % (N*T))

# Correlations. =====================================

# Maximum lag time.
max_lag_time=2.5*R/c_syn

In [ ]:
df=np.float(n)/T
print('sampling rate = %g Hz' % df)

In [ ]:
# Source positions.
i = np.arange(0, nsrc, dtype=float) + 0.5
phi = np.arccos(1 - 2*i/nsrc)
goldenRatio = (1 + 5**0.5)/2
theta = 2 * np.pi * i / goldenRatio
x_src, y_src, z_src = R_src*(np.cos(theta) * np.sin(phi)), R_src*(np.sin(theta) * np.sin(phi)), R_src*np.cos(phi);

# Receiver positions.
phi_rec=np.arange(0.0,2.0*np.pi,np.pi)
x_rec=R*np.cos(phi_rec)
y_rec=R*np.sin(phi_rec)
z_rec=np.zeros(len(x_rec))

# Plot geometry.
fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x_src,y_src,z_src,c='k')
ax.scatter(x_rec,y_rec,z_rec,c='r',marker='^')
ax.set_xlabel('x [m]')
fig.tight_layout()
plt.show()

In [ ]:
# Time increment.
dt=T/np.float(n)
# Maximum lag index.
maxlag=int(max_lag_time/dt)

# Number of receivers.
nrec=len(phi_rec)

In [ ]:
def corr(a,b,maxlag):
    
    n=len(a)
    cc=np.zeros(maxlag)
    
    for i in range(maxlag): cc[i]=np.sum(a[:n-i]*b[i:])
        
    return cc

In [ ]:
# Linear correlation stack.
cc_syn=np.zeros((nrec,maxlag))
cc_obs=np.zeros((nrec,maxlag))
# Phase-weights for phase-weighted stack.
pw_syn=np.zeros((nrec,maxlag),dtype='complex128')
pw_obs=np.zeros((nrec,maxlag),dtype='complex128')

# Loop over realisation.
for i in range(N):
    
    print('wavefield realisation %d' % i)
    u_syn=np.zeros((nrec,n))
    u_obs=np.zeros((nrec,n))
    
    # Make random wavefield. ==========================================================

    # Loop over sources.
    for isrc in range(nsrc):

        # Time series for one source.
        srcwvlt=np.random.randn(n)
        # Apply bandpass filter.
        srcwvlt=bandpass(srcwvlt,freqmin=freqmin,freqmax=freqmax,df=df,corners=4,zerophase=True)
        
        # Loop over receivers.
        for irec in range(nrec):

            # Source-receiver distance.
            r=np.sqrt((x_rec[irec]-x_src[isrc])**2+(y_rec[irec]-y_src[isrc])**2+(z_rec[irec]-z_src[isrc])**2)
            # Time shift.
            shift_t_syn=r/c_syn
            shift_t_obs=r/c_obs
            # Index shift.
            shift_n_syn=int(np.round(shift_t_syn/dt))
            shift_n_obs=int(np.round(shift_t_obs/dt))
            # Assign shifted time series to receiver.
            u_syn[irec,shift_n_syn:]+=srcwvlt[0:n-shift_n_syn]/r
            u_obs[irec,shift_n_obs:]+=srcwvlt[0:n-shift_n_obs]/r
            
    # Compute and stack correlations. =================================================
    for irec in range(1,nrec):
        # Linear stack.
        cc_i_syn=corr(u_syn[0,:],u_syn[irec,:],maxlag)
        cc_i_obs=corr(u_obs[0,:],u_obs[irec,:],maxlag)
        cc_syn[irec,:]+=cc_i_syn
        cc_obs[irec,:]+=cc_i_obs
        # Phase weights.
        h_syn=ss.hilbert(cc_i_syn)
        h_obs=ss.hilbert(cc_i_obs)
        pw_syn[irec,:]+=h_syn/np.abs(h_syn)
        pw_obs[irec,:]+=h_obs/np.abs(h_obs)
        
# Normalise.
cc_syn=cc_syn/np.max(cc_syn)
cc_obs=cc_obs/np.max(cc_obs)
pw_syn=np.abs(pw_syn/np.max(pw_syn))
pw_obs=np.abs(pw_obs/np.max(pw_obs))

In [ ]:
cc_syn_pw=(pw_syn[irec,:]**4.0)*cc_syn[1,:]
cc_obs_pw=(pw_obs[irec,:]**4.0)*cc_obs[1,:]

In [ ]:
# Receiver index.
irec=1

# Predicted traveltimes between receiver 0 and receiver irec.
tt_syn=1000*np.sqrt((x_rec[0]-x_rec[irec])**2+(y_rec[0]-y_rec[irec])**2+(z_rec[0]-z_rec[irec])**2)/c_syn
tt_obs=1000*np.sqrt((x_rec[0]-x_rec[irec])**2+(y_rec[0]-y_rec[irec])**2+(z_rec[0]-z_rec[irec])**2)/c_obs
print('predicted traveltime: %f ms' % tt_syn)

# Time axis [ms].
t=1000.0*np.linspace(0.0,np.float(maxlag)*dt,maxlag)

plt.subplots(1, figsize=(30,10))
plt.plot(t,cc_obs[irec,:],'k')
plt.plot(t,cc_syn[irec,:],'b')
plt.plot([tt_obs,tt_obs],[-0.1,1.1],'k')
plt.plot([tt_syn,tt_syn],[-0.1,1.1],'b')
plt.xlabel('t [ms]')
plt.grid()
plt.title('linear correlation stack')
plt.show()

plt.subplots(1, figsize=(30,10))
plt.plot(t,pw_obs[irec,:],'k')
plt.plot(t,pw_syn[irec,:],'b')
plt.plot([tt_obs,tt_obs],[-0.1,1.1],'k')
plt.plot([tt_syn,tt_syn],[-0.1,1.1],'b')
plt.xlabel('t [ms]')
plt.grid()
plt.title('phase weight')
plt.show()

plt.subplots(1, figsize=(30,10))
plt.plot(t,cc_obs_pw,'k')
plt.plot(t,cc_syn_pw,'b')
plt.plot([tt_obs,tt_obs],[-0.1,1.1],'k')
plt.plot([tt_syn,tt_syn],[-0.1,1.1],'b')
plt.xlabel('t [ms]')
plt.grid()
plt.title('phase-weighted correlation stack')
plt.show()

In [ ]:
# Analytical traveltime difference. =====================================

dtt_analytic=tt_syn-tt_obs

# Traveltime difference by waveform correlation. ========================

# Interpolate to increase time resolution.
f_syn=interpolate.interp1d(t,cc_syn_pw,kind='cubic')
f_obs=interpolate.interp1d(t,cc_obs_pw,kind='cubic')

t_interp=1000.0*np.linspace(0.0,np.float(maxlag)*dt,10*maxlag)
cc_syn_interp=f_syn(t_interp)
cc_obs_interp=f_obs(t_interp)

# Define measurement window.
T=0.5/freqmin
t_min=tt_syn-1000.0*T/1.0
t_max=tt_syn+1000.0*T/1.0
win=np.array([t_interp>t_min]).astype(float)*np.array([t_interp<t_max]).astype(float)
win=win[0]

plt.subplots(1, figsize=(30,10))
plt.plot(t_interp,win*cc_syn_interp,'b')
plt.plot(t_interp,win*cc_obs_interp,'k')
plt.show()

cc=corr(win*cc_obs_interp,win*cc_syn_interp,2000)
tt=np.linspace(0.0,2000.0*(t_interp[1]-t_interp[0]),2000)

dtt_numeric=np.float(tt[np.max(cc)==cc])

plt.subplots(1, figsize=(30,10))
plt.plot(tt,cc)
plt.xlabel('t [ms]')
plt.grid()
plt.show()

# Results. ===============================================================

print('analytical time shift: %g ms' % dtt_analytic)
print('numerical time shift: %g ms' % dtt_numeric)
ddtt=dtt_analytic-dtt_numeric
print('absolute time shift error: %g ms' % ddtt )
ddtt_rel=100.0*ddtt/tt_syn
print('relative time shift error: %g percent' % ddtt_rel)
dv=-c_syn*ddtt/tt_syn
print('velocity error: %g m/s' % dv)